In [2]:
import polars as pl

# ❶ 샘플만 읽기 (앞에서부터 n_rows개) – 필요하면 n_rows 크기는 조절
SAMPLE_ROWS = 100_000  # 예시: 10만 행만 샘플로 사용

scan_sample = pl.scan_parquet(
    "output2.parquet",
    n_rows=SAMPLE_ROWS,
)


In [3]:
scan_all = pl.scan_parquet("output2.parquet")


In [4]:
#import polars as pl

# 모든 컬럼 다 보여줘
#pl.Config.set_tbl_cols(-1)

# 보고 싶은 행 수 (예: 100개까지만, 전부 다는 -1)
#pl.Config.set_tbl_rows(100)   # or -1


### 1. 날짜 관계 파악 및 이상치 검토

결측치 확인

In [5]:
missing_tokens = [
    "None",
    "^UNK",
    "NOT APPLICABLE",
    "N/A",
    "NA",
]

def check_missing_ratio(lf: pl.LazyFrame, col: str):
    """
    lf: LazyFrame (여기서는 scan_sample)
    col: 컬럼 이름
    반환: 샘플에서 이 컬럼의 '가짜 결측 + 공백' 비율
    """
    result = (
        lf
        .select(
            pl.col(col).cast(pl.Utf8).alias("col")
        )
        .select([
            pl.col("col")
            .is_in(missing_tokens)
            .alias("is_token_missing"),

            pl.col("col")
            .str.strip_chars()
            .eq("")
            .alias("is_blank")
        ])
        .select((
            (pl.col("is_token_missing") | pl.col("is_blank"))
            .cast(pl.Int8)
            .mean()
            .alias("missing_ratio")
        ))
    )

    # 👉 여기서만 collect (원칙 2)
    return result.collect()


In [6]:
check_missing_ratio(scan_sample, "date_received")
check_missing_ratio(scan_sample, "date_added")
check_missing_ratio(scan_sample, "date_changed")


missing_ratio
f64
0.0


샘플에서 파싱이 얼마나 잘되는지

In [7]:
def check_date_parse_success(lf: pl.LazyFrame, col: str, fmt: str = "%Y%m%d"):
    """
    해당 컬럼을 주어진 포맷으로 Date로 파싱했을 때,
    샘플에서 몇 %가 성공하는지 확인.
    """
    result = (
        lf
        .select(pl.col(col).cast(pl.Utf8).alias("raw"))
        .with_columns(
            pl.col("raw")
            .str.strptime(pl.Date, format=fmt, strict=False)
            .alias("parsed")
        )
        .select([
            (pl.col("parsed").is_not_null()).cast(pl.Int8).mean().alias("parse_success_ratio")
        ])
    )

    return result.collect()


In [8]:
check_date_parse_success(scan_sample, "date_received")
check_date_parse_success(scan_sample, "date_added")
check_date_parse_success(scan_sample, "date_changed")


parse_success_ratio
f64
1.0


날짜간 관계 형태 파악
-> 음수 있는지 확인 (음수는 이상치)

In [9]:
date_cols = ["date_received", "date_added", "date_changed"]

scan_dates_sample = (
    scan_sample
    .select(date_cols)
    .drop_nulls()
    .with_columns(
        [
            pl.col("date_received").cast(pl.Utf8).str.strptime(pl.Date, format="%Y%m%d", strict=False),
            pl.col("date_added").cast(pl.Utf8).str.strptime(pl.Date, format="%Y%m%d", strict=False),
            pl.col("date_changed").cast(pl.Utf8).str.strptime(pl.Date, format="%Y%m%d", strict=False),
        ]
    )
    .with_columns(
        [
            (
                pl.col("date_added").cast(pl.Int64)
                - pl.col("date_received").cast(pl.Int64)
            ).alias("days_added_minus_received"),

            (
                pl.col("date_changed").cast(pl.Int64)
                - pl.col("date_added").cast(pl.Int64)
            ).alias("days_changed_minus_added"),

            (
                pl.col("date_changed").cast(pl.Int64)
                - pl.col("date_received").cast(pl.Int64)
            ).alias("days_changed_minus_received"),
        ]
    )
)

date_diff_summary_sample = scan_dates_sample.select(
    [
        pl.col("days_added_minus_received").min().alias("min_add_rec"),
        pl.col("days_added_minus_received").median().alias("med_add_rec"),
        pl.col("days_added_minus_received").mean().alias("mean_add_rec"),

        pl.col("days_changed_minus_added").min().alias("min_chg_add"),
        pl.col("days_changed_minus_added").median().alias("med_chg_add"),
        pl.col("days_changed_minus_added").mean().alias("mean_chg_add"),

        pl.col("days_changed_minus_received").min().alias("min_total"),
        pl.col("days_changed_minus_received").median().alias("med_total"),
        pl.col("days_changed_minus_received").mean().alias("mean_total"),
    ]
)

date_diff_summary_sample.collect()


min_add_rec,med_add_rec,mean_add_rec,min_chg_add,med_chg_add,mean_chg_add,min_total,med_total,mean_total
i64,f64,f64,i64,f64,f64,i64,f64,f64
0,0.0,0.06966,0,49.0,80.34224,0,49.0,80.4119


In [10]:
def summarize_date_gaps(
    lf: pl.LazyFrame,
    recv_col: str = "date_received",
    add_col: str = "date_added",
    chg_col: str = "date_changed",
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    return (
        lf
        .select([recv_col, add_col, chg_col])
        .drop_nulls()
        .with_columns([
            pl.col(recv_col)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(recv_col),
            pl.col(add_col)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(add_col),
            pl.col(chg_col)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(chg_col),
        ])
        .with_columns([
            (pl.col(add_col).cast(pl.Int64) - pl.col(recv_col).cast(pl.Int64))
                .alias("days_add_minus_recv"),
            (pl.col(chg_col).cast(pl.Int64) - pl.col(add_col).cast(pl.Int64))
                .alias("days_chg_minus_add"),
            (pl.col(chg_col).cast(pl.Int64) - pl.col(recv_col).cast(pl.Int64))
                .alias("days_chg_minus_recv"),
        ])
        .select([
            pl.col("days_add_minus_recv").min().alias("min_add_rec"),
            pl.col("days_add_minus_recv").median().alias("med_add_rec"),
            pl.col("days_add_minus_recv").mean().alias("mean_add_rec"),

            pl.col("days_chg_minus_add").min().alias("min_chg_add"),
            pl.col("days_chg_minus_add").median().alias("med_chg_add"),
            pl.col("days_chg_minus_add").mean().alias("mean_chg_add"),

            pl.col("days_chg_minus_recv").min().alias("min_total"),
            pl.col("days_chg_minus_recv").median().alias("med_total"),
            pl.col("days_chg_minus_recv").mean().alias("mean_total"),
        ])
    )


In [11]:
summarize_date_gaps(scan_sample).collect()


min_add_rec,med_add_rec,mean_add_rec,min_chg_add,med_chg_add,mean_chg_add,min_total,med_total,mean_total
i64,f64,f64,i64,f64,f64,i64,f64,f64
0,0.0,0.06966,0,49.0,80.34224,0,49.0,80.4119


### 2. 중복값 확인
#### 중복 있으면 안되는 컬럼:mdr_report_key

In [12]:
mdr_key_uniqueness_q = scan_sample.select(
    [
        pl.count().alias("total_rows"),
        pl.col("mdr_report_key").n_unique().alias("unique_mdr_report_key"),
    ]
)

mdr_key_uniqueness_q.collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\385391415.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("total_rows"),


total_rows,unique_mdr_report_key
u32,u32
100000,100000


함수화

In [13]:
def key_uniqueness_query(lf: pl.LazyFrame, key_col: str):
    """
    lf: LazyFrame (scan_parquet 결과)
    key_col: 고유해야 한다고 보는 컬럼 이름 (예: 'mdr_report_key')

    반환: total_rows, unique_keys 두 개 컬럼을 가진 LazyFrame 쿼리
    """
    q = lf.select(
        [
            pl.count().alias("total_rows"),
            pl.col(key_col).n_unique().alias("unique_keys"),
        ]
    )
    return q


In [14]:
# 샘플에서 먼저 감만 보기
key_uniqueness_query(scan_sample, "mdr_report_key").collect()

# 괜찮다 싶으면 전체에 대해 실행
# key_uniqueness_query(scan_full, "mdr_report_key").collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2696972248.py:10: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("total_rows"),


total_rows,unique_keys
u32,u32
100000,100000


중복값 확인으로 분포/품질 

확인한 컬럼 event_type, product_problem_flag, product_problems,device_0_openfda_device_class,pma_pmn_number

mdr_report_key → 중복이면 데이터가 잘못 들어온 거 의심 (유일성 체크 필수)

event_type → 사건의 분포 확인(death, injury, malfunction 등), 이상치/결측치 확인

product_problem_flag → 이진 레이블의 구조/불균형/이상값 확인
- 혹시나 타겟값이 될수도 있으니까..

product_problems → 주요 문제 유형 패턴 / 텍스트 코딩 품질 파악(같은 의미인데 다르게 기록된 건지)
- 카테고리나 군집 만들 때 기준이 될 수도 있음

device_*_openfda_device_class → 등급 정보가 깨끗한지(02, II, 이런 거 섞여있지 않은지) + 클래스별 데이터량 확인

pma_pmn_number → 어떤 기기가 반복적으로 문제를 많이 내는지 + 번호 코딩 품질 
- 특정 번호 하나에 많이 몰려있으면 그 기기 라인이 문제를 많이 내고 있다는 신호
- 리콜/이슈 히스토리랑 연결 가능 (사고 많이 난 번호 vs 실제 리콜된 번호 비교 가능)
- pma_pmn 넘버 같다고 완전한 동일 기기 아님(이건 내가 계속 헷갈려서)

In [15]:
def value_counts_query(lf: pl.LazyFrame, col: str, top_n: int = 20):
    q = (
        lf
        .group_by(col)
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .head(top_n)
    )
    return q


In [16]:
# 사건 타입 분포
value_counts_query(scan_sample, "event_type", top_n=20).collect()


event_type,count
str,u32
"""Malfunction""",58952
"""Injury""",40223
"""Death""",819
"""Other""",6


In [17]:
# product_problem_flag 이진 플래그 분포
value_counts_query(scan_sample, "product_problem_flag").collect()

product_problem_flag,count
str,u32
"""Y""",71258
"""N""",28742


In [18]:
# product_problems 상위 20개
value_counts_query(scan_sample, "product_problems", top_n=20).collect()

product_problems,count
str,u32
"""['Failure to Osseointegrate']""",10616
"""['Adverse Event Without Identi…",6644
"""['Wireless Communication Probl…",4856
"""['Loss of Osseointegration']""",3493
"""['Degraded']""",3398
…,…
"""['Obstruction of Flow']""",984
"""['Premature End-of-Life Indica…",927
"""['Unintended Application Progr…",910


In [19]:
# device class 분포
value_counts_query(scan_sample, "device_0_openfda_device_class").collect()

device_0_openfda_device_class,count
str,u32
"""2""",69087
"""3""",27053
"""1""",3474
"""U""",228
"""f""",106
"""N""",50
null,2


In [20]:
# 많이 문제된 PMA/510k 번호 TOP 20
value_counts_query(scan_sample, "pma_pmn_number", top_n=20).collect()

pma_pmn_number,count
str,u32
null,10203
"""DEN170088""",6389
"""EXEMPT""",4738
"""K213919""",4652
"""K203234""",4208
…,…
"""P930039""",1103
"""K032854""",1015
"""K203768""",964


### 3. categorical 컬럼 고윳값 개수, 비율 확인

확인할 컬럼
* adverse_event_flag
* mdr_text.text_type_code
* report_to_fda
* report_to_manufacturer
* event_location
* manufacturer_link_flag
* openfda_device_class


In [21]:
def categorical_profile_query(lf: pl.LazyFrame, col: str, top_n: int = 20):
    """
    lf: LazyFrame
    col: 분석할 컬럼
    top_n: 상위 몇 개 카테고리 보여줄지
    """

    # 전체 행 수
    total_rows_q = lf.select(pl.count().alias("total_rows"))

    # 고윳값 개수
    unique_values_q = lf.select(pl.col(col).n_unique().alias("unique_values"))

    # value_counts
    value_counts_q = (
        lf
        .group_by(col)
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .head(top_n)
    )

    # 세 정보(total_rows, unique_values, value_counts)를 함께 조합
    q = (
        value_counts_q
        .join(total_rows_q, how="cross")
        .join(unique_values_q, how="cross")
        .with_columns([
            (pl.col("count") / pl.col("total_rows")).alias("value_ratio"),
            (pl.col("unique_values") / pl.col("total_rows")).alias("unique_ratio")
        ])
        .select([col, "count", "value_ratio", "unique_ratio"])
    )

    return q


In [22]:
categorical_profile_query(
    scan_sample,
    "adverse_event_flag",
    top_n=20
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2601809448.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows_q = lf.select(pl.count().alias("total_rows"))


adverse_event_flag,count,value_ratio,unique_ratio
str,u32,f64,f64
"""N""",59173,0.59173,0.00003
"""Y""",40826,0.40826,0.00003
null,1,0.00001,0.00003


In [23]:
categorical_profile_query(
    scan_sample,
    "mdr_text_0_text_type_code",
    top_n=20
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2601809448.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows_q = lf.select(pl.count().alias("total_rows"))


mdr_text_0_text_type_code,count,value_ratio,unique_ratio
str,u32,f64,f64
"""Additional Manufacturer Narrat…",61105,0.61105,0.00003
"""Description of Event or Proble…",38890,0.3889,0.00003
null,5,0.00005,0.00003


In [24]:
categorical_profile_query(
    scan_sample,
    "report_to_fda",
    top_n=20
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2601809448.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows_q = lf.select(pl.count().alias("total_rows"))


report_to_fda,count,value_ratio,unique_ratio
str,u32,f64,f64
null,75798,0.75798,0.00004
"""N""",18447,0.18447,0.00004
"""Y""",5754,0.05754,0.00004
"""I""",1,0.00001,0.00004


In [25]:
categorical_profile_query(
    scan_sample,
    "report_to_manufacturer",
    top_n=20
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2601809448.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows_q = lf.select(pl.count().alias("total_rows"))


report_to_manufacturer,count,value_ratio,unique_ratio
str,u32,f64,f64
null,100000,1.0,0.00001


In [26]:
categorical_profile_query(
    scan_sample,
    "event_location",
    top_n=20
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2601809448.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows_q = lf.select(pl.count().alias("total_rows"))


event_location,count,value_ratio,unique_ratio
str,u32,f64,f64
null,89990,0.8999,0.00007
"""OTHER""",7437,0.07437,0.00007
"""OUTPATIENT TREATMENT FACILITY""",2295,0.02295,0.00007
"""HOSPITAL""",256,0.00256,0.00007
"""HOME""",19,0.00019,0.00007
"""NURSING HOME""",2,0.00002,0.00007
"""NO INFORMATION""",1,0.00001,0.00007


In [27]:
categorical_profile_query(
    scan_sample,
    "manufacturer_link_flag",
    top_n=20
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2601809448.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows_q = lf.select(pl.count().alias("total_rows"))


manufacturer_link_flag,count,value_ratio,unique_ratio
str,u32,f64,f64
"""Y""",96306,0.96306,0.00002
"""N""",3694,0.03694,0.00002


In [28]:
categorical_profile_query(
    scan_sample,
    "device_0_openfda_device_class",
    top_n=20
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2601809448.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  total_rows_q = lf.select(pl.count().alias("total_rows"))


device_0_openfda_device_class,count,value_ratio,unique_ratio
str,u32,f64,f64
"""2""",69087,0.69087,0.00007
"""3""",27053,0.27053,0.00007
"""1""",3474,0.03474,0.00007
"""U""",228,0.00228,0.00007
"""f""",106,0.00106,0.00007
"""N""",50,0.0005,0.00007
null,2,0.00002,0.00007


### 4. device.brand_name 등록번호와 매칭 후 (같은 기기인지 확인 후) 최빈값 대체

* 실제로 등록번호와 매칭이 맞는지 검토
시각화에 사용

등록번호 - 브랜드 조합별 카운트 계산 (샘플기준)

In [29]:
# 등록번호 - 브랜드 조합 카운트 함수 
def reg_brand_counts_query(
    lf: pl.LazyFrame,
    reg_col: str,
    brand_col: str,
):
    """
    등록번호(reg_col) - 브랜드(brand_col) 조합별 count 계산 쿼리.
    collect()는 밖에서 호출.
    """
    q = (
        lf
        .select([
            pl.col(reg_col).alias("reg_no"),
            pl.col(brand_col).alias("brand_name"),
        ])
        .drop_nulls()
        .group_by(["reg_no", "brand_name"])
        .agg(pl.len().alias("count"))
    )
    return q


In [30]:
# 샘플만 확인
reg_brand_counts = reg_brand_counts_query(
    scan_sample,
    "device_0_openfda_registration_number",
    "device_0_brand_name",
)

reg_brand_counts.collect().head(20)


reg_no,brand_name,count
str,str,u32
"""['3005099803']""","""AXIOS STENT AND ELECTROCAUTERY…",9
"""['9610617']""","""INNER SHEATH FOR 27050 SC""",1
"""['3004936110']""","""CARDIOMEMS PATIENT ELECTRONIC …",16
"""['2210968']""","""VISTASEAL DUAL APPL 3X""",4
"""['3008766073']""","""1.5T LINX, 17 BEADS""",3
…,…,…
"""['9610773']""","""TELESCOPE, 10 MM, 30°, HD, QUI…",5
"""['1818910']""","""ATTUNE CR RP INSRT SZ 6 6MM""",1
"""['1028232']""","""PK PAPYRUS US 2.5/20""",1


In [31]:
# 등록번호별 최빈 brand_name + 품질지표 함수
def reg_brand_mode_query(
    lf: pl.LazyFrame,
    reg_col: str,
    brand_col: str,
):
    """
    등록번호별:
      - 최빈 brand_name
      - 그 브랜드 count
      - 등록번호 전체 건수(total_for_reg)
      - 최빈 브랜드 비율(top_brand_ratio)
      - 브랜드 개수(n_brands)
    를 계산하는 쿼리.
    """

    reg_brand_counts = reg_brand_counts_query(lf, reg_col, brand_col)

    q = (
        reg_brand_counts
        .with_columns([
            # 등록번호별 전체 건수
            pl.col("count").sum().over("reg_no").alias("total_for_reg"),

            # 등록번호별 서로 다른 브랜드 개수
            pl.count().over("reg_no").alias("n_brands"),

            # 등록번호별 count 기준 랭킹 (1 = 최빈 브랜드)
            pl.col("count")
              .rank("dense", descending=True)
              .over("reg_no")
              .alias("rank_in_reg"),
        ])
        # 최빈 브랜드만 남기기
        .filter(pl.col("rank_in_reg") == 1)
        .with_columns([
            (pl.col("count") / pl.col("total_for_reg")).alias("top_brand_ratio")
        ])
        .select([
            "reg_no",
            "brand_name",
            "count",
            "total_for_reg",
            "top_brand_ratio",
            "n_brands",
        ])
    )

    return q


In [32]:
# 샘플만 확인
reg_brand_mode = reg_brand_mode_query(
    scan_sample,
    "device_0_openfda_registration_number",
    "device_0_brand_name",
)

reg_brand_mode.collect().head(20)


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3419629422.py:26: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().over("reg_no").alias("n_brands"),


reg_no,brand_name,count,total_for_reg,top_brand_ratio,n_brands
str,str,u32,u32,f64,u32
"""['2023826']""","""IMPLANTABLE COLLAMER LENS (ICL…",228,235,0.970213,5
"""['2184009']""","""ACT PLUS INSTRUMENT""",7,11,0.636364,5
"""['9610773']""","""VIDEO-OPTIK ""ENDOEYE 3D"", 30°""",25,65,0.384615,12
"""['1627487']""","""KIT IMPLANTABLE SLIM TIP LEAD,…",50,64,0.78125,4
"""['3006232063']""","""BARRICAID ACD""",1,1,1.0,1
…,…,…,…,…,…
"""['9615010']""","""OPMI VARIO""",1,1,1.0,1
"""['3002809144']""","""ALINITY I SYPHILIS TP REAGENT …",3,5,0.6,2
"""['3006425876']""","""EXTRACTION BAG FOR MIS""",1,1,1.0,1


In [33]:
# 믿을만한 매핑만 보고싶을 때
def stable_reg_brand_mode_query(
    lf: pl.LazyFrame,
    reg_col: str,
    brand_col: str,
    min_top_ratio: float = 0.8,
    require_single_brand: bool = False,
):
    """
    등록번호-브랜드 모드 중에서 '믿을 만한' 매핑만 필터링하는 쿼리.

    - min_top_ratio: 최빈 브랜드 비율 기준 (기본 0.8)
    - require_single_brand: True면 n_brands == 1 인 등록번호만 남김
    """

    base = reg_brand_mode_query(lf, reg_col, brand_col)

    if require_single_brand:
        q = base.filter(
            (pl.col("n_brands") == 1) &
            (pl.col("top_brand_ratio") >= min_top_ratio)
        )
    else:
        q = base.filter(
            pl.col("top_brand_ratio") >= min_top_ratio
        )

    return q


In [34]:
# 샘플만 확인
# 1) 최빈 브랜드 비율 0.8 이상인 등록번호들
stable_mode = stable_reg_brand_mode_query(
    scan_sample,
    "device_0_openfda_registration_number",
    "device_0_brand_name",
    min_top_ratio=0.8,
    require_single_brand=False,
)

stable_mode.collect().head(20)

# 2) 아예 브랜드가 1개뿐인 등록번호들만 (가장 안전)
stable_mode_strict = stable_reg_brand_mode_query(
    scan_sample,
    "device_0_openfda_registration_number",
    "device_0_brand_name",
    min_top_ratio=0.8,
    require_single_brand=True,
)

stable_mode_strict.collect().head(20)


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3419629422.py:26: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().over("reg_no").alias("n_brands"),


reg_no,brand_name,count,total_for_reg,top_brand_ratio,n_brands
str,str,u32,u32,f64,u32
"""['8040412']""","""NASOPHARYNGEAL AIRWAY, LATEX F…",2,2,1.0,1
"""['3004426659']""","""NEUROPACE RNS SYSTEM""",1,1,1.0,1
"""['9612515']""","""THORAFLEX HYBRID""",3,3,1.0,1
"""['1526439']""","""MOD BENDER TIP-B, 5.5MM, 5 DEG""",1,1,1.0,1
"""['1722028']""","""SPECTRA OPTIA""",7,7,1.0,1
…,…,…,…,…,…
"""['2032600']""","""PROCLEIX ULTRIO ELITE ASSAY""",2,2,1.0,1
"""['9615010']""","""OPMI VARIO""",1,1,1.0,1
"""['3005168196']""","""INDIGO SYSTEM CATRX ASPIRATION…",2,2,1.0,1


In [35]:
# 검토용으로 특정 번호 한 개 골라서 브랜드 분포 직접 보는 함수
def reg_brand_distribution_query(
    lf: pl.LazyFrame,
    reg_col: str,
    brand_col: str,
    reg_no_value: str,
):
    """
    특정 등록번호(reg_no_value)에 대해
    브랜드별 count 분포를 보는 쿼리.
    """

    q = (
        reg_brand_counts_query(lf, reg_col, brand_col)
        .filter(pl.col("reg_no") == reg_no_value)
        .sort("count", descending=True)
    )
    return q


In [36]:
# 샘플만 확인
# 먼저 샘플에서 의심되는 등록번호 하나 뽑고
suspect_reg = (
    reg_brand_mode_query(
        scan_sample,
        "device_0_openfda_registration_number",
        "device_0_brand_name",
    )
    .filter(pl.col("top_brand_ratio") < 0.8)
    .select("reg_no")
    .head(1)
    .collect()["reg_no"][0]
)

# 그 등록번호에 대해 브랜드 분포 확인
reg_brand_distribution_query(
    scan_sample,
    "device_0_openfda_registration_number",
    "device_0_brand_name",
    suspect_reg,
).collect()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3419629422.py:26: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().over("reg_no").alias("n_brands"),


reg_no,brand_name,count
str,str,u32
"""['2015691']""","""EVOQUE TRICUSPID VALVE REPLACE…",8
"""['2015691']""","""EDWARDS SAPIEN 3 ULTRA TRANSCA…",3
"""['2015691']""","""COMMANDER DELIVERY SYSTEM""",1
"""['2015691']""","""FORESIGHT ELITE SENSOR LARGE""",1
"""['2015691']""","""EDWARDS COMMANDER DELIVERY SYS…",1
"""['2015691']""","""EDWARDS SAPIEN 3 ULTRA RESILIA…",1
"""['2015691']""","""CO-SET PLUS FLOW-TRU HOUSING""",1
"""['2015691']""","""SWAN-GANZ CCOMBO V CCO/SVO2/CE…",1


adverse_event_flag

In [37]:
# 어떤 값 있는지 보기
adv_flag_counts_sample = (
    scan_sample
    .select(
        pl.col("adverse_event_flag").value_counts()
    )
)

adv_flag_counts_sample.collect()


adverse_event_flag
struct[2]
"{""N"",59173}"
"{null,1}"
"{""Y"",40826}"


In [38]:
row_adv_null_subset = (
    scan_sample
    .filter(pl.col("adverse_event_flag").is_null())
    .select([
        "mdr_report_key",
        "event_type",
        "product_problems",
        "date_received",
        "adverse_event_flag",
    ])
)

row_adv_null_subset.collect()


mdr_report_key,event_type,product_problems,date_received,adverse_event_flag
str,str,str,str,str
"""19656582""","""Malfunction""","""['Device Operational Issue']""","""20240702""",null


In [39]:
adv_flag_counts_all = (
    scan_all
    .select(
        pl.col("adverse_event_flag").value_counts()
    )
)

adv_flag_counts_all.collect()  # 출력할 때만 collect


adverse_event_flag
struct[2]
"{null,78}"
"{""N"",1296468}"
"{""Y"",950604}"


In [40]:
rows_adv_null_subset_all = (
    scan_all
    .filter(pl.col("adverse_event_flag").is_null())
    .select([
        "product_problem_flag",
        "mdr_report_key",
        "event_type",
        "date_received",
        "adverse_event_flag",
    ])
)

rows_adv_null_subset_all.collect().to_pandas()


,product_problem_flag,mdr_report_key,event_type,date_received,adverse_event_flag
0,Y,19656582,Malfunction,20240702,None
1,N,19656958,No answer provided,20240701,None
2,N,19910639,No answer provided,20240805,None
3,Y,19933292,Malfunction,20240807,None
4,N,20282833,No answer provided,20240920,None
...,...,...,...,...,...
73,N,18990010,No answer provided,20240328,None
74,Y,18833166,Malfunction,20240304,None
75,N,18706634,No answer provided,20240214,None
76,N,18562519,Other,20240118,None


In [41]:
(
    scan_all
    .filter(pl.col("adverse_event_flag").is_null())
    .select(
        pl.col("event_type").value_counts()
    )
).collect().to_pandas()


,event_type
0,"{'event_type': 'Other', 'count': 2}"
1,"{'event_type': 'No answer provided', 'count': 46}"
2,"{'event_type': 'Malfunction', 'count': 30}"


adverse_event_flag가 널값이면
event_type이 거의 malfunction 또는 no answer provided

product_problem_flag

In [42]:
# (
#     scan_all
#     .select(
#         pl.col("event_type").unique().alias("event_type_unique")
#    )
# ).collect()


In [43]:
adv_flag_counts_all = (
    scan_all
    .select(
        pl.col("product_problem_flag").value_counts()
    )
)

adv_flag_counts_all.collect() 

product_problem_flag
struct[2]
"{""N"",785342}"
"{""Y"",1461808}"


In [44]:
date_parse_summary = (
    scan_all
    .select([
        pl.len().alias("n_total"),

        # date_of_event 파싱 성공 개수
        pl.col("date_of_event")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .is_not_null()
          .sum()
          .alias("n_parsed_date_of_event"),

        # date_received 파싱 성공 개수
        pl.col("date_received")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .is_not_null()
          .sum()
          .alias("n_parsed_date_received"),

        # device_date_of_manufacturer 파싱 성공 개수
        pl.col("device_date_of_manufacturer")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .is_not_null()
          .sum()
          .alias("n_parsed_device_date_of_manufacturer"),
    ])
    .with_columns([
        (pl.col("n_parsed_date_of_event") / pl.col("n_total")).alias("ratio_parsed_date_of_event"),
        (pl.col("n_parsed_date_received") / pl.col("n_total")).alias("ratio_parsed_date_received"),
        (pl.col("n_parsed_device_date_of_manufacturer") / pl.col("n_total")).alias("ratio_parsed_device_date_of_manufacturer"),
    ])
)

date_parse_summary.collect().to_pandas()


,n_total,n_parsed_date_of_event,n_parsed_date_received,n_parsed_device_date_of_manufacturer,ratio_parsed_date_of_event,ratio_parsed_date_received,ratio_parsed_device_date_of_manufacturer
0,2247150,2061761,2247150,1595407,0.9175,1.0,0.709969


In [45]:
import polars as pl

date_quality_3 = (
    scan_all
    .select([
        pl.len().alias("n_rows"),

        # 1) date_of_event
        pl.col("date_of_event").is_null().sum().alias("event_n_original_null"),
        pl.col("date_of_event")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .is_not_null()
          .sum()
          .alias("event_n_parsed"),

        # 2) date_received
        pl.col("date_received").is_null().sum().alias("recv_n_original_null"),
        pl.col("date_received")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .is_not_null()
          .sum()
          .alias("recv_n_parsed"),

        # 3) device_date_of_manufacturer
        pl.col("device_date_of_manufacturer").is_null().sum().alias("mfr_n_original_null"),
        pl.col("device_date_of_manufacturer")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .is_not_null()
          .sum()
          .alias("mfr_n_parsed"),
    ])
    .with_columns([
        # 원본 null 비율
        (pl.col("event_n_original_null") / pl.col("n_rows")).alias("event_ratio_original_null"),
        (pl.col("recv_n_original_null") / pl.col("n_rows")).alias("recv_ratio_original_null"),
        (pl.col("mfr_n_original_null") / pl.col("n_rows")).alias("mfr_ratio_original_null"),

        # (null 아닌 것 중) 파싱 성공 비율
        (
            pl.col("event_n_parsed")
            / (pl.col("n_rows") - pl.col("event_n_original_null"))
        ).alias("event_ratio_parsed_given_nonnull"),

        (
            pl.col("recv_n_parsed")
            / (pl.col("n_rows") - pl.col("recv_n_original_null"))
        ).alias("recv_ratio_parsed_given_nonnull"),

        (
            pl.col("mfr_n_parsed")
            / (pl.col("n_rows") - pl.col("mfr_n_original_null"))
        ).alias("mfr_ratio_parsed_given_nonnull"),
    ])
)

# 출력할 때만 collect
date_quality_3.collect().to_pandas()


,n_rows,event_n_original_null,event_n_parsed,recv_n_original_null,recv_n_parsed,mfr_n_original_null,mfr_n_parsed,event_ratio_original_null,recv_ratio_original_null,mfr_ratio_original_null,event_ratio_parsed_given_nonnull,recv_ratio_parsed_given_nonnull,mfr_ratio_parsed_given_nonnull
0,2247150,185389,2061761,0,2247150,651743,1595407,0.0825,0.0,0.290031,1.0,1.0,1.0


date_of_event, date_received, device_date_of_manufacturer 날짜 차이 확인
* event: 이상사건이 처음 발생한 날짜
* received: fda가 해당 보고 접수한 날짜
* manufacturer: 의료기기 제조 날짜

정상 순서
* mfr - event - received

In [46]:
date_gaps_all = (
    scan_all
    .select([
        "date_of_event",
        "date_received",
        "device_date_of_manufacturer",
    ])
    .drop_nulls()  # 세 날짜 다 있는 행만 대상으로 순서/차이 확인
    .with_columns([
        pl.col("date_of_event")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .alias("date_of_event"),

        pl.col("date_received")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .alias("date_received"),

        pl.col("device_date_of_manufacturer")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=False)
          .alias("device_date_of_manufacturer"),
    ])
    .with_columns([
        # 사건일 - 제조일 (정상이라면 >= 0이어야 함)
        (pl.col("date_of_event").cast(pl.Int64) - pl.col("device_date_of_manufacturer").cast(pl.Int64))
            .alias("days_event_minus_mfr"),

        # 수신일 - 사건일 (정상이라면 >= 0)
        (pl.col("date_received").cast(pl.Int64) - pl.col("date_of_event").cast(pl.Int64))
            .alias("days_recv_minus_event"),

        # 수신일 - 제조일 (정상이라면 >= 0)
        (pl.col("date_received").cast(pl.Int64) - pl.col("device_date_of_manufacturer").cast(pl.Int64))
            .alias("days_recv_minus_mfr"),
    ])
)


In [47]:
date_gap_summary = (
    date_gaps_all
    .select([
        # 사건일 - 제조일
        pl.col("days_event_minus_mfr").min().alias("min_event_mfr"),
        pl.col("days_event_minus_mfr").median().alias("med_event_mfr"),
        pl.col("days_event_minus_mfr").mean().alias("mean_event_mfr"),
        pl.col("days_event_minus_mfr").max().alias("max_event_mfr"),

        # 수신일 - 사건일
        pl.col("days_recv_minus_event").min().alias("min_recv_event"),
        pl.col("days_recv_minus_event").median().alias("med_recv_event"),
        pl.col("days_recv_minus_event").mean().alias("mean_recv_event"),
        pl.col("days_recv_minus_event").max().alias("max_recv_event"),

        # 수신일 - 제조일
        pl.col("days_recv_minus_mfr").min().alias("min_recv_mfr"),
        pl.col("days_recv_minus_mfr").median().alias("med_recv_mfr"),
        pl.col("days_recv_minus_mfr").mean().alias("mean_recv_mfr"),
        pl.col("days_recv_minus_mfr").max().alias("max_recv_mfr"),
    ])
)

date_gap_summary.collect().to_pandas()


,min_event_mfr,med_event_mfr,mean_event_mfr,max_event_mfr,min_recv_event,med_recv_event,mean_recv_event,max_recv_event,min_recv_mfr,med_recv_mfr,mean_recv_mfr,max_recv_mfr
0,-44540,433.0,1160.049045,45572,-551,26.0,85.939925,45123,-427,508.0,1245.98897,45651


In [48]:
date_order_violations = (
    date_gaps_all
    .select([
        # 사건이 제조보다 전에 일어난 것 (말이 안 됨)
        (pl.col("days_event_minus_mfr") < 0).sum().alias("n_event_before_mfr"),

        # FDA가 사건보다 먼저 받은 것 (말이 안 됨)
        (pl.col("days_recv_minus_event") < 0).sum().alias("n_recv_before_event"),

        # FDA가 제조일보다 먼저 받은 것 (이것도 말이 안 됨)
        (pl.col("days_recv_minus_mfr") < 0).sum().alias("n_recv_before_mfr"),
    ])
)

date_order_violations.collect().to_pandas()


,n_event_before_mfr,n_recv_before_event,n_recv_before_mfr
0,6611,1087,382


In [49]:
date_order_violations = (
    date_gaps_all
    .select([
        pl.len().alias("n_rows"),

        # 사건이 제조보다 전에 일어난 것 (말이 안 됨)
        (pl.col("days_event_minus_mfr") < 0).sum().alias("n_event_before_mfr"),

        # FDA가 사건보다 먼저 받은 것 (말이 안 됨)
        (pl.col("days_recv_minus_event") < 0).sum().alias("n_recv_before_event"),

        # FDA가 제조일보다 먼저 받은 것 (이것도 말이 안 됨)
        (pl.col("days_recv_minus_mfr") < 0).sum().alias("n_recv_before_mfr"),
    ])
    .with_columns([
        (pl.col("n_event_before_mfr") / pl.col("n_rows")).alias("ratio_event_before_mfr"),
        (pl.col("n_recv_before_event") / pl.col("n_rows")).alias("ratio_recv_before_event"),
        (pl.col("n_recv_before_mfr") / pl.col("n_rows")).alias("ratio_recv_before_mfr"),
    ])
)

date_order_violations.collect().to_pandas()


,n_rows,n_event_before_mfr,n_recv_before_event,n_recv_before_mfr,ratio_event_before_mfr,ratio_recv_before_event,ratio_recv_before_mfr
0,1534951,6611,1087,382,0.004307,0.000708,0.000249


previous_use_code  문제된 기기가 처음 사용된 것인지, 재사용된 것인지, 또는 알 수 없는지

In [50]:
null_like_regex = r"(?i)^(None|UNK.*|NOT REPORTED|NOT APPLICABLE|N/A|NA|-|\*)$"


In [51]:
null_like_regex = r"(?i)^(None|UNK.*|NOT REPORTED|NOT APPLICABLE|N/A|NA|-|\*)$"

expr_effective_null = (
    pl.col("previous_use_code")
      .cast(pl.Utf8)
      .str.replace_all(r"\s+", " ")      # 공백 정규화
      .str.strip_chars()                 # 앞뒤 공백 제거
      .str.contains(null_like_regex, literal=False)   # ← match 대신 contains
)


In [52]:
(
    scan_all
    .select([
        "previous_use_code",
        expr_effective_null.alias("is_effective_null")
    ])
).collect().head()


previous_use_code,is_effective_null
str,bool
"""I""",false
"""A""",false
"""I""",false
"""I""",false
"""I""",false


In [53]:
# 진짜 널값 찾는 함수
import polars as pl

def summarize_effective_null_ratio(lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    """
    col에 대해:
      - n_rows: 전체 행 수
      - n_physical_null: 진짜 null 개수
      - n_placeholder_null: UNK, NOT REPORTED, N/A, 공백 등 '텍스트 결측' 개수
      - n_effective_null: 위 둘을 합친 '실질 결측' 개수
      - ratio_physical_null
      - ratio_placeholder_null
      - ratio_effective_null
    를 LazyFrame으로 반환 (collect는 호출하는 쪽에서!)
    """

    # 네가 말한 패턴들 전부 포함 + 공백/빈 문자열
    null_like_regex = r"(?i)^(None|UNK.*|NOT REPORTED|NOT APPLICABLE|N/A|NA|-|\*)$|^\s*$"

    placeholder_expr = (
        pl.col(col)
          .cast(pl.Utf8)
          .str.replace_all(r"\s+", " ")   # 공백 여러 개 → 하나
          .str.strip_chars()              # 앞뒤 공백 제거
          .str.contains(null_like_regex, literal=False)
    )

    base = (
        lf
        .select([
            pl.len().alias("n_rows"),

            # 1) 물리적 null
            pl.col(col).is_null().sum().alias("n_physical_null"),

            # 2) placeholder 결측
            placeholder_expr.sum().alias("n_placeholder_null"),
        ])
        .with_columns([
            # 3) 실질 결측 개수 = 둘 합
            (pl.col("n_physical_null") + pl.col("n_placeholder_null")).alias("n_effective_null"),
        ])
        .with_columns([
            (pl.col("n_physical_null") / pl.col("n_rows")).alias("ratio_physical_null"),
            (pl.col("n_placeholder_null") / pl.col("n_rows")).alias("ratio_placeholder_null"),
            (pl.col("n_effective_null") / pl.col("n_rows")).alias("ratio_effective_null"),
        ])
    )

    return base


In [54]:
prev_use_null_summary = summarize_effective_null_ratio(
    scan_all,
    "previous_use_code",
)

prev_use_null_summary.collect()


n_rows,n_physical_null,n_placeholder_null,n_effective_null,ratio_physical_null,ratio_placeholder_null,ratio_effective_null
u32,u32,u32,u32,f64,f64,f64
2247150,148872,964,149836,0.066249,0.000429,0.066678


In [55]:
prev_use_null_summary = summarize_effective_null_ratio(
    scan_all,
    "date_of_event",
)

prev_use_null_summary.collect()

n_rows,n_physical_null,n_placeholder_null,n_effective_null,ratio_physical_null,ratio_placeholder_null,ratio_effective_null
u32,u32,u32,u32,f64,f64,f64
2247150,185389,0,185389,0.0825,0.0,0.0825


In [56]:
prev_use_null_summary = summarize_effective_null_ratio(
    scan_all,
    "single_use_flag",
)

prev_use_null_summary.collect()

n_rows,n_physical_null,n_placeholder_null,n_effective_null,ratio_physical_null,ratio_placeholder_null,ratio_effective_null
u32,u32,u32,u32,f64,f64,f64
2247150,176727,0,176727,0.078645,0.0,0.078645


In [57]:
scan_all.columns


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\793295413.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  scan_all.columns


['adverse_event_flag',
 'date_added',
 'date_changed',
 'date_facility_aware',
 'date_manufacturer_received',
 'date_of_event',
 'date_received',
 'date_report',
 'date_report_to_fda',
 'date_report_to_manufacturer',
 'device',
 'device_0_brand_name',
 'device_0_catalog_number',
 'device_0_combination_product_flag',
 'device_0_date_received',
 'device_0_date_removed_flag',
 'device_0_date_returned_to_manufacturer',
 'device_0_device_age_text',
 'device_0_device_availability',
 'device_0_device_evaluated_by_manufacturer',
 'device_0_device_event_key',
 'device_0_device_operator',
 'device_0_device_report_product_code',
 'device_0_device_sequence_number',
 'device_0_expiration_date_of_device',
 'device_0_generic_name',
 'device_0_implant_flag',
 'device_0_lot_number',
 'device_0_manufacturer_d_address_1',
 'device_0_manufacturer_d_address_2',
 'device_0_manufacturer_d_city',
 'device_0_manufacturer_d_country',
 'device_0_manufacturer_d_name',
 'device_0_manufacturer_d_postal_code',
 'dev

In [58]:
cols = scan_all.columns
len(cols)


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\302816398.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  cols = scan_all.columns


354

In [59]:
print("\n".join(scan_all.columns))


adverse_event_flag
date_added
date_changed
date_facility_aware
date_manufacturer_received
date_of_event
date_received
date_report
date_report_to_fda
date_report_to_manufacturer
device
device_0_brand_name
device_0_catalog_number
device_0_combination_product_flag
device_0_date_received
device_0_date_removed_flag
device_0_date_returned_to_manufacturer
device_0_device_age_text
device_0_device_availability
device_0_device_evaluated_by_manufacturer
device_0_device_event_key
device_0_device_operator
device_0_device_report_product_code
device_0_device_sequence_number
device_0_expiration_date_of_device
device_0_generic_name
device_0_implant_flag
device_0_lot_number
device_0_manufacturer_d_address_1
device_0_manufacturer_d_address_2
device_0_manufacturer_d_city
device_0_manufacturer_d_country
device_0_manufacturer_d_name
device_0_manufacturer_d_postal_code
device_0_manufacturer_d_state
device_0_manufacturer_d_zip_code
device_0_manufacturer_d_zip_code_ext
device_0_model_number
device_0_openfda_de

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\531014347.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print("\n".join(scan_all.columns))


openfda_device_class

In [60]:
scan_all.select(
    pl.col("device_0_openfda_device_class").unique().sort()
).collect()


device_0_openfda_device_class
str
null
"""1"""
"""2"""
"""3"""
"""N"""
"""U"""
"""f"""


In [61]:
import polars as pl

openfda_vc = (
    scan_all
    .group_by("device_0_openfda_device_class")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()


device_0_openfda_device_class,len,ratio
str,u32,f64
"""2""",1693548,0.753643
"""3""",492244,0.219053
"""1""",52635,0.023423
"""U""",4698,0.002091
"""N""",2492,0.001109
"""f""",1496,0.000666
null,37,0.000016


단일 컬럼을 categorical로 변환

In [62]:
# CATEGORICAL 타입 변환 함수
def cast_to_categorical(lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    """
    단일 컬럼을 Categorical로 변환
    """
    if col not in lf.columns:
        print(f"[WARN] Column '{col}' not found. Skipped.")
        return lf


    return lf.with_columns(
        pl.col(col).cast(pl.Categorical)
    )


In [63]:
scan_all = cast_to_categorical(scan_all, "device_0_openfda_device_class")


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3275987326.py:6: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col not in lf.columns:


_openfda_device_class'로 끝나는 모든 컬럼을 Categorical로 변환

In [64]:
import polars as pl

def cast_device_openfda_classes_to_cat(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    이름이 '*_openfda_device_class'로 끝나는 모든 컬럼을 Categorical로 변환
    예: device_0_openfda_device_class, device_1_openfda_device_class, ...
    """
    # 1) 컬럼 이름 중에서 패턴 맞는 것만 골라오기
    target_cols = [
        col for col in lf.columns
        if col.endswith("_openfda_device_class")
    ]

    if not target_cols:
        return lf  # 해당 컬럼이 없으면 그대로 반환

    # 2) 골라낸 컬럼들을 전부 Categorical로 캐스팅
    exprs = [pl.col(col).cast(pl.Categorical) for col in target_cols]

    return lf.with_columns(exprs)


In [65]:
scan_all = pl.scan_parquet("output2.parquet")

scan_all = cast_device_openfda_classes_to_cat(scan_all)

scan_all.collect().head()


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\176736271.py:10: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  col for col in lf.columns


adverse_event_flag,date_added,date_changed,date_facility_aware,date_manufacturer_received,date_of_event,date_received,date_report,date_report_to_fda,date_report_to_manufacturer,device,device_0_brand_name,device_0_catalog_number,device_0_combination_product_flag,device_0_date_received,device_0_date_removed_flag,device_0_date_returned_to_manufacturer,device_0_device_age_text,device_0_device_availability,device_0_device_evaluated_by_manufacturer,device_0_device_event_key,device_0_device_operator,device_0_device_report_product_code,device_0_device_sequence_number,device_0_expiration_date_of_device,device_0_generic_name,device_0_implant_flag,device_0_lot_number,device_0_manufacturer_d_address_1,device_0_manufacturer_d_address_2,device_0_manufacturer_d_city,device_0_manufacturer_d_country,device_0_manufacturer_d_name,device_0_manufacturer_d_postal_code,device_0_manufacturer_d_state,device_0_manufacturer_d_zip_code,device_0_manufacturer_d_zip_code_ext,…,patient_2_patient_age,patient_2_patient_ethnicity,patient_2_patient_race,patient_2_patient_sequence_number,patient_2_patient_sex,patient_2_patient_weight,patient_2_sequence_number_outcome,patient_2_sequence_number_treatment,patient_3_date_received,patient_3_patient_age,patient_3_patient_ethnicity,patient_3_patient_race,patient_3_patient_sequence_number,patient_3_patient_sex,patient_3_patient_weight,patient_3_sequence_number_outcome,patient_3_sequence_number_treatment,pma_pmn_number,previous_use_code,product_problem_flag,product_problems,remedial_action,removal_correction_number,report_date,report_number,report_source_code,report_to_fda,report_to_manufacturer,reporter_country_code,reporter_occupation_code,reprocessed_and_reused_flag,single_use_flag,source_type,summary_report_flag,suppl_dates_fda_received,suppl_dates_mfr_received,type_of_report
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""N""","""20240506""","""20240715""",null,"""20240408""","""20240406""","""20240506""","""20240603""",null,null,null,"""DEXCOM G6 CONTINUOUS GLUCOSE M…","""STT-GS-003""","""Y""","""20240506""",null,null,"""DA""","""Yes""","""N""",null,"""LAY USER/PATIENT""","""QBJ""","""1""","""20240303""","""CONTINUOUS GLUCOSE MONITOR""",null,"""5324339""","""6340 SEQUENCE DR.""",null,"""SAN DIEGO""","""US""","""DEXCOM, INC.""","""92121""","""CA""","""92121""",null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""DEN170088""","""I""","""Y""","""['No Device Output']""",null,null,null,"""3004753838-2024-104727""","""Manufacturer report""",null,null,"""FR""","""OTHER""","""N""","""N""","""['Foreign', 'Distributor']""","""N""","""06/03/2024""","""05/09/2024""","""['Initial submission', 'Follow…"
"""N""","""20240509""","""20240906""",null,"""20240422""","""20240421""","""20240509""","""20240906""",null,null,null,"""PUMP MMT-1780KPK 670G PATHWAY …","""MMT-1780KPK""","""N""","""20240509""",null,null,"""DA""","""Device was returned to manufac…","""Y""",null,"""LAY USER/PATIENT""","""OZP""","""1""","""20250522""","""AUTOMATED INSULIN DOSING DEVIC…",null,"""HG68QU5""","""CEIBA NORTE IND. PARK #50 ROAD""",null,"""JUNCOS""",null,"""MEDTRONIC PUERTO RICO OPERATIO…","""00777-3869""",null,"""00777""","""-386""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""P150001""","""A""","""Y""","""['Power Problem']""",null,null,null,"""2032227-2024-166432""","""Manufacturer report""",null,null,"""US""","""003""","""N""","""N""","""['Consumer']""","""N""","""09/06/2024""","""08/13/2024""","""['Initial submission', 'Follow…"
"""N""","""20240514""","""20240617""",null,"""20240512""","""20240512""","""20240514""","""20240514""",null,null,null,"""OMNIPOD 5 POD""","""POD-BLE-H1-520""","""N""","""20240514""",null,null,

In [66]:
# 타입 변환 잘 됐는지 확인
{c: scan_all.schema[c] for c in [
    "device_0_openfda_device_class",
    "adverse_event_flag",
    "product_problem_flag",
    "date_of_event",
]}



C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2198707821.py:2: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'device_0_openfda_device_class': Categorical,
 'adverse_event_flag': String,
 'product_problem_flag': String,
 'date_of_event': String}

adverse_evnet_flag, product_problem_flag
* boolean 타입 변환

In [67]:
openfda_vc = (
    scan_all
    .group_by("adverse_event_flag")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

adverse_event_flag,len,ratio
str,u32,f64
"""N""",1296468,0.576939
"""Y""",950604,0.423027
null,78,0.000035


In [68]:
openfda_vc = (
    scan_all
    .group_by("product_problem_flag")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

product_problem_flag,len,ratio
str,u32,f64
"""Y""",1461808,0.650516
"""N""",785342,0.349484


In [69]:
def cast_flags_to_bool(
    lf: pl.LazyFrame,
    flag_cols: list[str] = ["adverse_event_flag", "product_problem_flag"],
) -> pl.LazyFrame:
    return lf.with_columns([
        pl.when(pl.col(col) == "Y")
          .then(pl.lit(True))
          .when(pl.col(col) == "N")
          .then(pl.lit(False))
          .otherwise(None)
          .alias(col)
        for col in flag_cols
        if col in lf.columns
    ])


In [70]:
scan_all = pl.scan_parquet("output2.parquet")

scan_all = cast_flags_to_bool(scan_all)

scan_all.schema


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\188062363.py:13: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col in lf.columns
C:\Users\USER\AppData\Local\Temp\ipykernel_25100\501681266.py:5: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  scan_all.schema


Schema([('adverse_event_flag', Boolean),
        ('date_added', String),
        ('date_changed', String),
        ('date_facility_aware', String),
        ('date_manufacturer_received', String),
        ('date_of_event', String),
        ('date_received', String),
        ('date_report', String),
        ('date_report_to_fda', String),
        ('date_report_to_manufacturer', String),
        ('device', String),
        ('device_0_brand_name', String),
        ('device_0_catalog_number', String),
        ('device_0_combination_product_flag', String),
        ('device_0_date_received', String),
        ('device_0_date_removed_flag', String),
        ('device_0_date_returned_to_manufacturer', String),
        ('device_0_device_age_text', String),
        ('device_0_device_availability', String),
        ('device_0_device_evaluated_by_manufacturer', String),
        ('device_0_device_event_key', String),
        ('device_0_device_operator', String),
        ('device_0_device_report_produc

In [71]:
{c: scan_all.schema[c] for c in [
    "adverse_event_flag",
    "product_problem_flag"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3235062700.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'adverse_event_flag': Boolean, 'product_problem_flag': Boolean}

In [72]:
openfda_vc = (
    scan_all
    .group_by("adverse_event_flag")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

adverse_event_flag,len,ratio
bool,u32,f64
false,1296468,0.576939
true,950604,0.423027
null,78,0.000035


In [73]:
openfda_vc = (
    scan_all
    .group_by("product_problem_flag")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

product_problem_flag,len,ratio
bool,u32,f64
true,1461808,0.650516
false,785342,0.349484


날짜컬럼들 타입 변환

In [74]:
{c: scan_all.schema[c] for c in [
    "date_of_event",
    "date_received",
    "device_date_of_manufacturer"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2169447266.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'date_of_event': String,
 'date_received': String,
 'device_date_of_manufacturer': String}

날짜 컬럼 원시 포맷 검사 단계

In [75]:
date_cols = [
    "date_of_event",
    "date_received",
    "device_date_of_manufacturer",
]


길이가 8자리가 맞는지, 다른 포맷 섞여있는지, 빈문자열이 null이외에 unk이런 거 있는지 확인하는 함수

In [76]:
def summarize_date_format(lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    """
    날짜 컬럼 원시 포맷 요약:
    - 문자열 길이 (str_len)
    - 8자리 숫자(YYYYMMDD) 여부 (is_8_digit)
    - '-' / '/' 포함 여부 (has_dash, has_slash)
    """
    if col not in lf.columns:
        raise ValueError(f"컬럼 '{col}'이(가) lf에 없음")

    return (
        lf
        .select(
            pl.col(col)
              .cast(pl.Utf8)
              .alias(col)
        )
        .with_columns([
            pl.col(col).str.len_chars().alias("str_len"),
            pl.col(col).str.contains(r"^\d{8}$").alias("is_8_digit"),
            pl.col(col).str.contains("-").alias("has_dash"),
            pl.col(col).str.contains("/").alias("has_slash"),
        ])
        .group_by(["str_len", "is_8_digit", "has_dash", "has_slash"])
        .len()
        .rename({"len": "n_rows"})        # 개수 컬럼 이름 변경
        .sort("n_rows", descending=True)  # 많이 나온 조합부터 보기
    )


In [77]:
#summarize_date_format(scan_all, "date_of_event").collect() # 널값 깔끔 나머지도 불순물 없음
#summarize_date_format(scan_all, "date_received").collect() # 널값 없음, 포맷 깔끔
summarize_date_format(scan_all, "device_date_of_manufacturer").collect() # 포맷 깔끔


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\1841933295.py:8: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col not in lf.columns:


str_len,is_8_digit,has_dash,has_slash,n_rows
u32,bool,bool,bool,u32
8,true,false,false,1595407
null,null,null,null,651743


널값 형태 확인

In [78]:
openfda_vc = (
    scan_all
    .group_by("date_of_event")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

date_of_event,len,ratio
str,u32,f64
null,185389,0.0825
"""20231012""",16800,0.007476
"""20240101""",16550,0.007365
"""20231009""",15320,0.006818
"""20240701""",12097,0.005383
…,…,…
"""20100329""",1,4.4501e-7
"""20040928""",1,4.4501e-7
"""19881013""",1,4.4501e-7


In [79]:
# 5년치 돌릴 때는 샘플 검사 먼저 해보기
(
    scan_all
    .select(
        pl.col("date_of_event")
          .cast(pl.Utf8)
          .str.strptime(pl.Date, format="%Y%m%d", strict=True)
    )
    .limit(100_000)
).collect()


date_of_event
date
2024-04-06
2024-04-21
2024-05-12
2024-04-28
2022-02-09
…
2024-07-28
2024-08-01
null


데이트타입으로 변환하는 함수

In [80]:
def cast_date_cols(
    lf: pl.LazyFrame,
    date_cols: list[str],
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    """
    YYYYMMDD 형식의 날짜 컬럼들을 pl.Date로 변환하는 함수.
    - strict=False: 파싱 실패 값은 null로 떨어짐.
    """
    exprs = []
    for col in date_cols:
        if col in lf.columns:
            exprs.append(
                pl.col(col)
                  .cast(pl.Utf8)                                # 1) 문자열로 캐스팅
                  .str.strptime(pl.Date, format=fmt, strict=False)  # 2) 여기서 Date로 파싱
                  .alias(col)
            )

    if not exprs:
        return lf

    return lf.with_columns(exprs)


In [81]:
import polars as pl

scan_all = pl.scan_parquet("output2.parquet")

date_cols = [
    "date_of_event",
    "date_received",
    "device_date_of_manufacturer",
]

scan_all = cast_date_cols(scan_all, date_cols=date_cols)


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\530960774.py:12: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col in lf.columns:


In [82]:
{c: scan_all.schema[c] for c in [
    "date_of_event",
    "date_received",
    "device_date_of_manufacturer"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2169447266.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'date_of_event': Date,
 'date_received': Date,
 'device_date_of_manufacturer': Date}

In [83]:
openfda_vc = (
    scan_all
    .group_by("event_type")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect().to_pandas()

,event_type,len,ratio
0,Malfunction,1282468,0.570709
1,Injury,951128,0.423260
2,Death,13156,0.005855
3,Other,352,0.000157
4,No answer provided,46,0.000020


device_age_text 단위 맞추기

In [84]:
(
    scan_all
    .select("device_0_device_age_text")
    .limit(20)
).collect()

device_0_device_age_text
str
"""DA"""
"""DA"""
null
null
"""DA"""
…
null
null
"""DA"""


In [85]:
device_age_dist = (
    scan_all
    .group_by("device_0_device_age_text")
    .len()
    .sort("len", descending=True)
    .limit(30)      # 상위 30개 정도만
)

device_age_dist.collect()


device_0_device_age_text,len
str,u32
null,1361356
"""DA""",761197
"""MO""",4102
"""7 MO""",1867
"""6 MO""",1832
…,…
"""2 MO""",529
"""22 MO""",471
"""23 MO""",441


아니 이걸 단위를어떻게맞춰 ㅅㅣㅂ..

report_to_fda boolean 타입 변환하기
* 보고자가 해당 사건을 fda에게 보고했는지 여부

In [86]:
openfda_vc = (
    scan_all
    .group_by("report_to_fda")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

report_to_fda,len,ratio
str,u32,f64
null,1590062,0.707591
"""N""",451259,0.200814
"""Y""",205751,0.091561
"""I""",78,0.000035


In [87]:
(
    scan_all
    .filter(pl.col("report_to_fda") == "I")
    .select([
        "report_to_fda",
        "date_received",
        "manufacturer_link_flag"
    ])
    .limit(20)
).collect()


report_to_fda,date_received,manufacturer_link_flag
str,date,str
"""I""",2024-07-02,"""Y"""
"""I""",2024-07-01,"""Y"""
"""I""",2024-08-05,"""Y"""
"""I""",2024-08-07,"""Y"""
"""I""",2024-09-20,"""Y"""
…,…,…
"""I""",2024-08-30,"""Y"""
"""I""",2024-09-10,"""Y"""
"""I""",2024-07-24,"""Y"""


걍 널처리 하겠씀!!!

product_problem_flag도 변환

In [88]:
def cast_flags_to_bool(
    lf: pl.LazyFrame,
    flag_cols: list[str] = ["report_to_fda", "product_problem_flag", "adverse_event_flag"],
) -> pl.LazyFrame:
    return lf.with_columns([
        pl.when(pl.col(col) == "Y")
          .then(pl.lit(True))
          .when(pl.col(col) == "N")
          .then(pl.lit(False))
          .otherwise(None)
          .alias(col)
        for col in flag_cols
        if col in lf.columns
    ])

In [89]:
scan_all = pl.scan_parquet("output2.parquet")

scan_all = cast_flags_to_bool(scan_all)

scan_all.schema

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\439692767.py:13: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col in lf.columns
C:\Users\USER\AppData\Local\Temp\ipykernel_25100\1388968625.py:5: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  scan_all.schema


Schema([('adverse_event_flag', Boolean),
        ('date_added', String),
        ('date_changed', String),
        ('date_facility_aware', String),
        ('date_manufacturer_received', String),
        ('date_of_event', String),
        ('date_received', String),
        ('date_report', String),
        ('date_report_to_fda', String),
        ('date_report_to_manufacturer', String),
        ('device', String),
        ('device_0_brand_name', String),
        ('device_0_catalog_number', String),
        ('device_0_combination_product_flag', String),
        ('device_0_date_received', String),
        ('device_0_date_removed_flag', String),
        ('device_0_date_returned_to_manufacturer', String),
        ('device_0_device_age_text', String),
        ('device_0_device_availability', String),
        ('device_0_device_evaluated_by_manufacturer', String),
        ('device_0_device_event_key', String),
        ('device_0_device_operator', String),
        ('device_0_device_report_produc

In [90]:
{c: scan_all.schema[c] for c in [
    "report_to_fda"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2491074379.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'report_to_fda': Boolean}

device_age_text 단위 통일, 타입 변환

In [91]:
scan_all.select(pl.col("device_0_device_age_text").null_count()).collect()


device_0_device_age_text
u32
1361356


In [92]:
# 단위 있는/없는 것 구분
unit_pattern = r"(day|da|days|d|week|weeks|wk|wks|month|months|mo|year|years|yr|yrs)"
col = "device_0_device_age_text"  # 필요하면 여기 이름 바꿔

with_unit_flag = (
    scan_all
    .with_columns([
        pl.col(col).cast(pl.Utf8).str.to_lowercase().alias("age_text_lower"),
    ])
    .with_columns([
        pl.col("age_text_lower")
          .str.contains(unit_pattern)
          .alias("has_unit")
    ])
)


In [93]:
(
    with_unit_flag
    .filter(pl.col(col).is_not_null() & ~pl.col("has_unit"))
    .select(pl.len().alias("n_no_unit"))
).collect()


n_no_unit
u32
1


device_0_age_text 얘 단위들 어떻게 되어있는지 좀 봐야게씀!!

In [94]:
col = "device_0_device_age_text"

units = (
    scan_all
    .select([
        pl.col(col)
          .cast(pl.Utf8)
          .str.to_lowercase()
          .alias("age_text_lower")
    ])
    .with_columns([
        # 예: "3 days" → "days", "2 wk" → "wk"
        pl.col("age_text_lower")
          .str.extract(r"\d+\s*([a-zA-Z]+)", group_index=1)
          .alias("unit_token")
    ])
    .filter(pl.col("unit_token").is_not_null())
    .group_by("unit_token")
    .len()
    .sort("len", descending=True)
    .limit(50)   # 상위 50개만
)

units.collect()


unit_token,len
str,u32
"""da""",91315
"""mo""",28480
"""yr""",687
"""wk""",11


In [95]:
{c: scan_all.schema[c] for c in [
    "device_0_device_age_text"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\2474177139.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'device_0_device_age_text': String}

In [96]:
def normalize_device_age_to_days(
    lf: pl.LazyFrame,
    src_col: str = "device_0_device_age_text",
    dst_col: str = "device_0_device_age_days",
) -> pl.LazyFrame:
    return (
        lf.with_columns([
            pl.col(src_col).cast(pl.Utf8).str.to_lowercase().alias("_age_text_lower"),
        ])
        .with_columns([
            # 숫자 추출
            pl.col("_age_text_lower")
              .str.extract(r"(\d+(\.\d+)?)", group_index=1)
              .cast(pl.Float64)
              .alias("_age_value"),

            # 단위 추출
            pl.col("_age_text_lower")
              .str.extract(r"(day|da|days|d|week|weeks|wk|wks|month|months|mo|year|years|yr|yrs)", group_index=1)
              .alias("_age_unit"),
        ])
        .with_columns([
            pl.when(pl.col("_age_value").is_null())
              .then(None)  # 숫자 자체가 없으면 null
            .when(pl.col("_age_unit").is_null())
              .then(pl.col("_age_value"))  # 단위 없으면 day로 가정
            .when(pl.col("_age_unit").str.contains("day|days|d"))
              .then(pl.col("_age_value"))
            .when(pl.col("_age_unit").str.contains("week|weeks|wk|wks"))
              .then(pl.col("_age_value") * 7)
            .when(pl.col("_age_unit").str.contains("month|months|mo"))
              .then(pl.col("_age_value") * 30)
            .when(pl.col("_age_unit").str.contains("year|years|yr|yrs"))
              .then(pl.col("_age_value") * 365)
            .otherwise(None)
            .cast(pl.Int64)
            .alias(dst_col)
        ])
        .drop(["_age_text_lower", "_age_value", "_age_unit"])
    )


In [97]:
def normalize_all_device_age_cols_to_days(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    *_device_age_text 로 끝나는 모든 컬럼을 찾아서
    *_device_age_days (int, day 단위) 컬럼을 생성하는 함수.
    """
    # 1) 대상 컬럼들 찾기
    age_text_cols = [
        c for c in lf.columns
        if c.endswith("_device_age_text")
    ]

    # 2) 각 컬럼에 대해 normalize_device_age_to_days 적용
    for src_col in age_text_cols:
        dst_col = src_col.replace("_device_age_text", "_device_age_days")
        lf = normalize_device_age_to_days(lf, src_col=src_col, dst_col=dst_col)

    return lf


In [98]:
scan_all = scan_all.pipe(normalize_all_device_age_cols_to_days)


C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3415363299.py:8: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  c for c in lf.columns


In [99]:
{c: scan_all.schema[c] for c in [
    "device_0_device_age_days"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\160696970.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'device_0_device_age_days': Int64}

product_problem_flag × event_type
* product_problem_flag 보고서가 기기의 품질, 성능 또는 안전성과 관련된지 여부
* event_type 이상 사건과 관련된 결과 

In [100]:
{c: scan_all.schema[c] for c in [
    "product_problem_flag",
    "event_type"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3186593739.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'product_problem_flag': Boolean, 'event_type': String}

타입 변환부터 먼저 하기

In [101]:
# product_problem_flag 값 확인
openfda_vc = (
    scan_all
    .group_by("product_problem_flag")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

product_problem_flag,len,ratio
bool,u32,f64
true,1461808,0.650516
false,785342,0.349484


product_problem_flag boolean로 변환 위에서 했음

In [102]:
# event_type 값 확인
openfda_vc = (
    scan_all
    .group_by("event_type")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

event_type,len,ratio
str,u32,f64
"""Malfunction""",1282468,0.570709
"""Injury""",951128,0.42326
"""Death""",13156,0.005855
"""Other""",352,0.000157
"""No answer provided""",46,0.00002


In [103]:
scan_all = cast_to_categorical(scan_all, "event_type")

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3275987326.py:6: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if col not in lf.columns:


In [104]:
{c: scan_all.schema[c] for c in [
    "product_problem_flag",
    "event_type"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\3186593739.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'product_problem_flag': Boolean, 'event_type': Categorical}

event_type == "Malfunction"이면
product_problem_flag=True 이어야 맞음..

In [105]:
(
    scan_all
    .filter(
        (pl.col("event_type") == "Malfunction")
        & (pl.col("product_problem_flag") == False)
    )
    .select(pl.len().alias("n_inconsistent"))
).collect()


n_inconsistent
u32
4241


In [106]:
(
    scan_all
    .filter(
        (pl.col("event_type") == "Malfunction")
        & (pl.col("product_problem_flag") == False)
    )
    .select([
        "event_type",
        "product_problem_flag",
        "product_problems",
        "report_source_code",
        "device_0_brand_name",
        "device_0_model_number",
    ])
    .limit(1000)
).collect().to_pandas()


,event_type,product_problem_flag,product_problems,report_source_code,device_0_brand_name,device_0_model_number
0,Malfunction,False,['No Device Output'],Manufacturer report,"3100A VENT,115V,60HZ,DOM","3100A VENT,115V,60HZ,DOM"
1,Malfunction,False,"['Failure to Fire', 'Activation, Positioning o...",Manufacturer report,PADLOCK CLIP PRO-SELECT,C913131
2,Malfunction,False,['Migration'],Manufacturer report,MARINER MIS POLYAXIAL HEAD,MM1-000020
3,Malfunction,False,['Unexpected/Unintended Radiation Output'],Manufacturer report,PICOSURE,None
4,Malfunction,False,['Migration'],Manufacturer report,MARINER MIS POLYAXIAL HEAD,MM1-000020
...,...,...,...,...,...,...
995,Malfunction,False,['Unintended Application Program Shut Down'],Manufacturer report,DEXCOM G6 CONTINUOUS GLUCOSE MONITORING SYSTEM,None
996,Malfunction,False,['Improper or Incorrect Procedure or Method'],Manufacturer report,MARK 5 NUVO 10,595
997,Malfunction,False,['Insufficient Information'],Manufacturer report,"FLOWMETER, DIGITAL FLEXMOUNT",26289
998,Malfunction,False,['No Apparent Adverse Event'],Manufacturer report,UNKNOWN,INFUSION DEVICES - UNKNOWN SQD


In [107]:
openfda_vc = (
    scan_all
    .group_by("report_source_code")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

report_source_code,len,ratio
str,u32,f64
"""Manufacturer report""",2104501,0.93652
"""Distributor report""",125691,0.055934
"""Voluntary report""",11543,0.005137
"""User Facility report""",5415,0.00241


adverse_event_flag event_type과 비교하여 결측치 처리

In [108]:
{c: scan_all.schema[c] for c in [
    "adverse_event_flag",
    "event_type"
]}

C:\Users\USER\AppData\Local\Temp\ipykernel_25100\1370560539.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  {c: scan_all.schema[c] for c in [


{'adverse_event_flag': Boolean, 'event_type': Categorical}

In [109]:
openfda_vc = (
    scan_all
    .group_by("adverse_event_flag")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

adverse_event_flag,len,ratio
bool,u32,f64
false,1296468,0.576939
true,950604,0.423027
null,78,0.000035


In [110]:
# 이상 케이스 수 확인
weird_count = (
    scan_all
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & (pl.col("adverse_event_flag") == False)
    )
    .select(pl.len().alias("n_weird"))
)

weird_count.collect()


n_weird
u32
26146


In [111]:
weird_sample = (
    scan_all
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & (pl.col("adverse_event_flag") == False)
    )
    .select([
        "event_type",
        "adverse_event_flag",
        "report_source_code",
        "date_of_event",
        "date_received",
        "device_0_brand_name",
        "device_0_model_number",
    ])
    .limit(20)
)

weird_sample.collect()


event_type,adverse_event_flag,report_source_code,date_of_event,date_received,device_0_brand_name,device_0_model_number
cat,bool,str,str,str,str,str
"""Injury""",false,"""Manufacturer report""","""20240430""","""20240528""","""OUTBACK""",null
"""Injury""",false,"""Manufacturer report""","""20240314""","""20240409""","""PU MDP-60K DISP PUNCH 6.0MM""",null
"""Injury""",false,"""Manufacturer report""",null,"""20240404""","""SONATA""","""SONATA"""
"""Injury""",false,"""Manufacturer report""","""20240509""","""20240610""","""LIPOASPIRATE WASH SYSTEM""","""VIALITY-1400"""
"""Injury""",false,"""Voluntary report""",null,"""20240510""","""G6 SENSOR""",null
…,…,…,…,…,…,…
"""Injury""",false,"""Manufacturer report""","""20240317""","""20240408""","""T:SLIM X2 INSULIN PUMP WITH IN…","""1002717"""
"""Injury""",false,"""Manufacturer report""","""20240306""","""20240401""","""T:SLIM X2 INSULIN PUMP WITH IN…","""1002717"""
"""Injury""",false,"""Manufacturer report""","""20240318""","""20240403""","""PUMP MMT-1885A OUS BLE PUMP MM…","""MMT-1885A"""


In [112]:
# 결측치 케이스: Death/Injury 인데 flag가 null
weird_null_count = (
    scan_all
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & pl.col("adverse_event_flag").is_null()
    )
    .select(pl.len().alias("n_null"))
)

weird_null_count.collect()

weird_null_sample = (
    scan_all
    .filter(
        pl.col("event_type").is_in(["Death", "Injury"])
        & pl.col("adverse_event_flag").is_null()
    )
    .select([
        "mdr_report_key",
        "event_type",
        "adverse_event_flag",
        "report_source_code",
        "date_of_event",
        "date_received",
        "device_0_brand_name",
        "device_0_model_number",
    ])
    .limit(20)
)

weird_null_sample.collect()


mdr_report_key,event_type,adverse_event_flag,report_source_code,date_of_event,date_received,device_0_brand_name,device_0_model_number
str,cat,bool,str,str,str,str,str


In [114]:
openfda_vc = (
    scan_all
    .group_by("device_0_device_report_product_code")
    .len()
    .with_columns(
        (pl.col("len") / pl.col("len").sum()).alias("ratio")
    )
    .sort("len", descending=True)
)

openfda_vc.collect()

device_0_device_report_product_code,len,ratio
str,u32,f64
"""DZE""",580682,0.258408
"""QBJ""",302569,0.134646
"""QFG""",224485,0.099898
"""OZP""",111124,0.049451
"""BZD""",52697,0.023451
…,…,…
"""MRV""",1,4.4501e-7
"""PRZ""",1,4.4501e-7
"""OIV""",1,4.4501e-7


In [115]:
prev_use_null_summary = summarize_effective_null_ratio(
    scan_all,
    "device_0_device_report_product_code",
)

prev_use_null_summary.collect()

n_rows,n_physical_null,n_placeholder_null,n_effective_null,ratio_physical_null,ratio_placeholder_null,ratio_effective_null
u32,u32,u32,u32,f64,f64,f64
2247150,25,1,26,0.000011,4.4501e-7,0.000012


In [116]:
prev_use_null_summary = summarize_effective_null_ratio(
    scan_all,
    "date_report",
)

prev_use_null_summary.collect()

n_rows,n_physical_null,n_placeholder_null,n_effective_null,ratio_physical_null,ratio_placeholder_null,ratio_effective_null
u32,u32,u32,u32,f64,f64,f64
2247150,330806,0,330806,0.147211,0.0,0.147211
